In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mochammadrevan","key":"9825ed2f9a56b872ccd8af0b66caaec6"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d cahyaalkahfi/obesitas

Dataset URL: https://www.kaggle.com/datasets/cahyaalkahfi/obesitas
License(s): unknown
  0% 0.00/57.9k [00:00<?, ?B/s]
100% 57.9k/57.9k [00:00<00:00, 1.11MB/s]


In [4]:
!mkdir obesitas
!unzip obesitas.zip -d obesitas
!ls obesitas

Archive:  obesitas.zip
  inflating: obesitas/obesitas.csv   
obesitas.csv


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [6]:
df = pd.read_csv('obesitas/obesitas.csv')

In [7]:
df.head(10)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight
5,Male,29.0,1.62,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Automobile,Normal
6,Female,23.0,1.50,55.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,0.0,Sometimes,Motorbike,Normal
7,Male,22.0,1.64,53.0,no,no,2.0,3.0,Sometimes,no,2.0,no,3.0,0.0,Sometimes,Public_Transportation,Normal
8,Male,24.0,1.78,64.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Frequently,Public_Transportation,Normal
9,Male,22.0,1.72,68.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,no,Public_Transportation,Normal


In [23]:
df.NObeyesdad.value_counts()

NObeyesdad
2    972
3    580
1    287
0    272
Name: count, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [9]:
fitur_terpilih = ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF']
x = df[fitur_terpilih]
y = 'NObeyesdad'

In [10]:
label_encoders = {}

for kolom in df.select_dtypes(include=['object']).columns:
    if kolom in fitur_terpilih or kolom == 'NObeyesdad':
        label_encoders[kolom] = LabelEncoder()
        df[kolom] = label_encoders[kolom].fit_transform(df[kolom])

In [11]:
x = df[fitur_terpilih]
y = df['NObeyesdad']

In [12]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [14]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [15]:
num_classes = y_train.shape[1]

In [16]:
print("Dimensi X_train:", X_train.shape)
print("Dimensi y_train:", y_train.shape)
print("Dimensi X_test:", X_test.shape)
print("Dimensi y_test:", y_test.shape)

Dimensi X_train: (1688, 7)
Dimensi y_train: (1688, 4)
Dimensi X_test: (423, 7)
Dimensi y_test: (423, 4)


In [17]:
model = Sequential()
model.add(Dense(32, input_dim=len(fitur_terpilih), activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [18]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
43/43 [==============================] - 3s 14ms/step - loss: 1.2731 - accuracy: 0.4556 - val_loss: 1.1112 - val_accuracy: 0.6479
Epoch 2/50
43/43 [==============================] - 0s 5ms/step - loss: 1.0438 - accuracy: 0.6252 - val_loss: 0.8775 - val_accuracy: 0.6864
Epoch 3/50
43/43 [==============================] - 0s 4ms/step - loss: 0.8640 - accuracy: 0.6763 - val_loss: 0.7263 - val_accuracy: 0.7396
Epoch 4/50
43/43 [==============================] - 0s 4ms/step - loss: 0.7390 - accuracy: 0.7170 - val_loss: 0.6242 - val_accuracy: 0.7751
Epoch 5/50
43/43 [==============================] - 0s 4ms/step - loss: 0.6419 - accuracy: 0.7807 - val_loss: 0.5448 - val_accuracy: 0.8284
Epoch 6/50
43/43 [==============================] - 0s 4ms/step - loss: 0.5607 - accuracy: 0.8252 - val_loss: 0.4767 - val_accuracy: 0.8521
Epoch 7/50
43/43 [==============================] - 0s 4ms/step - loss: 0.4941 - accuracy: 0.8615 - val_loss: 0.4243 - val_accuracy: 0.8787
Epoch 8/50
43/43 [=

In [20]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Akurasi:", accuracy)

14/14 [==============================] - 0s 2ms/step - loss: 0.0774 - accuracy: 0.9740
Akurasi: 0.9739952683448792


In [21]:
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)
y_test_classes = y_test.argmax(axis=1)

14/14 [==============================] - 0s 1ms/step


In [22]:
print("Laporan Klasifikasi:\n", classification_report(y_test_classes, y_pred_classes))

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96        56
           1       0.98      0.87      0.92        62
           2       0.99      0.99      0.99       199
           3       0.96      0.98      0.97       106

    accuracy                           0.97       423
   macro avg       0.96      0.96      0.96       423
weighted avg       0.97      0.97      0.97       423



In [25]:
keras_file = "obesity.h5"
tf.keras.models.save_model(model, keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open("obesity.tflite","wb").write(tfmodel)

<ipython-input-25-1f01af7cbd43>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file)


5500